# Pre-processing of NeuroImaging data for structural connectivity analyses

The pre-processing steps of MRI data are necessary steps. These steps are largely non-trivial and should follow a clear path which can grow exponentially complex depending on the modality you are working with. In this notebook you will find a guideline to pre-process T1 and diffusion images before computing tractograms and structural connectivity matrices. There are plenty of resources online to learn and stay up-to-date with the recommended guidelines (e.g., [MRtrix3 - DWI tutorial](https://mrtrix.readthedocs.io/en/latest/dwi_preprocessing/denoising.html) or [Andy's Brain Book](https://andysbrainbook.readthedocs.io/en/latest/index.html)). Otherwise, you can always rely on asking the community questions and/or doubts (e.g., [Neurstars](https://neurostars.org/)). Below you can find an example of how you would cite and explain to the scientific community the steps that were followed to pre-process your dataset for further analyses.

*High resolution anatomical T1 weighted images were skull stripped [1], corrected for bias field inhomogeneities [2], registered to MNI space [3] and segmented into 5 tissue-type images [4]. Diffusion weighted images suffer from many artifacts all of which were appropriately corrected. Images were also skull-stripped [1], corrected for susceptibility-induced distortions [5], denoised [6], freed from Gibbs ringing artifacts [7] and corrected for eddy-currents and motion artifacts [8]. The preprocessed images were then co-registered to its corresponding anatomical template (already in MNI space) [3], resampled to a 1.5 mm3 voxel size and eventually corrected for bias field inhomogeneities [2]. After motion correction as well as registration to the MNI template, the B-matrix was appropriately rotated [9].*

[1] M. Jenkinson, M. Pechaud, S. Smith and others, “BET2: MR-based estimation of brain, skull and scalp surfaces,” in Eleventh annual meeting of the organization for human brain mapping, 2005.

[2] N. J. Tustison, B. B. Avants, P. A. Cook, Y. Zheng, A. Egan, P. A. Yushkevich and J. C. Gee, “N4ITK: improved N3 bias correction,” IEEE transactions on medical imaging, vol. 29, p. 1310–1320, 2010.

[3] M. Jenkinson, P. Bannister, M. Brady and S. Smith, “Improved Optimization for the Robust and Accurate Linear Registration and Motion Correction of Brain Images,” vol. 17, pp. 825-841, 2002.

[4] R. E. Smith, J.-D. Tournier, F. Calamante and A. Connelly, “Anatomically-constrained tractography: improved diffusion MRI streamlines tractography through effective use of anatomical information.,” NeuroImage, vol. 62, no. 3, p. 1924–1938, September 2012.

[5] J. L. R. Andersson, S. Skare and J. Ashburner, “How to correct susceptibility distortions in spin-echo echo-planar images: application to diffusion tensor imaging.,” NeuroImage, vol. 20, no. 2, p. 870–888, October 2003.

[6] J. Veraart, E. Fieremans and D. S. Novikov, “Diffusion MRI noise mapping using random matrix theory.,” Magnetic resonance in medicine, vol. 76, no. 5, p. 1582–1593, November 2016.

[7] E. Kellner, B. Dhital, V. G. Kiselev and M. Reisert, “Gibbs-ringing artifact removal based on local subvoxel-shifts.,” Magnetic resonance in medicine, vol. 76, no. 5, p. 1574–1581, November 2016.

[8] J. L. R. Andersson and S. N. Sotiropoulos, “An integrated approach to correction for off-resonance effects and subject movement in diffusion MR imaging.,” NeuroImage, vol. 125, p. 1063–1078, January 2016.

[9] A. Leemans and D. K. Jones, “The B-matrix must be rotated when correcting for subject motion in DTI data.,” Magnetic resonance in medicine, vol. 61, no. 6, p. 1336–1349, June 2009.

In [16]:
import os
import glob
import shutil

## Pre-processing of anatomical T1w MRI

Eventhough it might seem a bit odd, we need to deal with structural data usually coming from T1 acquisitions. The reason is because delineating anatomical constraints on the diffusion data has been shown to improve the correspondance between real structural connectivity and the reconstructed by means of tractography (see ref. [4] in the introduction). There are several softwares avaialble for this, each one with its *pros* and *cons*. As usual, we provide an example code that follows the citation template shown in the introduction - although some slight differences exist due to software incompatibility. We start by first obtaining the bids directory with all the derivatives. Don't worry, we provided data in the Data4Labs already in the correct format. Otherwise, recall the exercises in Lab-1. 

For T1 data, [FSL](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FSL) or [ANTs](https://stnava.github.io/ANTs/) are the safest options. [MRtrix3](https://mrtrix.readthedocs.io/en/latest/) offers some wrappers of them that come in handy if the goal is to use the pre-processed data for diffusion studies. Online containarized resources do exist that are specificallt designed for user friendliness, at the expense of freedom to operate [Brainlife](https://brainlife.io/about/). All of them are good!

In [18]:
subjects = ["01", "02", "06" ,"07"]     # List of subjects in the dataset to pre-process
sessions = [                            # Sessions per subject - Each subject might have different ones
    [""],
    [""],
    ["2"],
    ["","2"]
]
# In this simple dataset we can manually annotate the subjects and sessions, but in a real case scenario 
#     you would use automated python or shell commands (i.e., "find" or "os.listdir()")

for s, sub in enumerate(subjects):
    for ss, ses in enumerate(sessions[s]):
        # It is good practice to report summaries, updates, errors and logs
        print(f"Processing sub-CON{sub} in ses-control{ses}:")
        print("---------------------------------------------")
        
        # Relevant directories per subject and session
        bids_original = f"../Data4Labs/BIDS-dir/sub-CON{sub}/ses-control{ses}/anat/"
        bids_derivatives = f"../Data4Labs/BIDS-dir/derivatives/sub-CON{sub}/ses-control{ses}/anat/"     
        if not os.path.exists(bids_derivatives):
            os.makedirs(bids_derivatives)
        
        # Original T1 file
        file = os.path.join(bids_original, f"sub-CON{sub}_ses-control{ses}_T1w.nii.gz")
        
        # Brain Extraction Tool
        masked_brain = os.path.join(bids_derivatives, f"sub-CON{sub}_ses-control{ses}_T1w_brain.nii.gz")
        os.system(f"bet {file} {masked_brain} -f .5 -g -.6")
        os.system(f"bet {masked_brain} {masked_brain} -f .35 -m")
        print("Brain extracted succesfully")

        # Registration to template
        reference = "../Data4Labs/Utils/MNI152_T1_1mm_brain.nii.gz"
        registered = os.path.join(bids_derivatives, f"sub-CON{sub}_ses-control{ses}_T1w_brain_space-MNI152.nii.gz")
        transformation = os.path.join(bids_derivatives, f"sub-CON{sub}_ses-control{ses}_T1w_brain_matrix2MNI152.txt")
        os.system(f"flirt -in {masked_brain} -ref {reference} -omat {transformation}")
        os.system(f"flirt -in {masked_brain} -ref {reference} -out {registered} -applyxfm -init {transformation}")
        print("Registered to MNI152 succesfully")

        # Tissue segmentation
        os.system(f"fast -n 4 -I 25 -W 25 -H .25 {registered}")
        print("Segmented tissues succesfully")

        # We apply an extra layer of segmentation for subcortical gray matter structures
        # TODO: first fsl https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FIRST/UserGuide
        subcorticalGM = os.path.join(bids_derivatives, f"sub-CON{sub}_ses-control{ses}_T1w_brain_space-MNI152_subcortical.nii.gz")
        os.system(f"run_first_all -i {registered} -o {subcorticalGM}")
        os.rename(
            f"{bids_derivatives}sub-CON{sub}_ses-control{ses}_T1w_brain_space-MNI152_subcortical_all_fast_firstseg.nii.gz",
            f"{bids_derivatives}sub-CON{sub}_ses-control{ses}_T1w_brain_space-MNI152_subcortex-GM.nii.gz"
        )
        # Let's clean the output files that we won't be needing for this
        to_delete = glob.glob(f"{bids_derivatives}*_subcortical*")
        for f in to_delete:
            try:
                os.remove(f)
            except:
                shutil.rmtree(f)
        to_delete = glob.glob(f"{bids_derivatives}*_to_std_sub*")
        for f in to_delete:
            os.remove(f)
        print("Subcortical structures segmented succesfully")
        print("+++++++++++++++++++++++++++++++++++++++++++++++")
        print("+++++++++++++++++++++++++++++++++++++++++++++++")

Processing sub-CON01 in ses-control:
---------------------------------------------
Brain extracted succesfully
Registered to MNI152 succesfully
Segmented tissues succesfully
Subcortical structures segmented succesfully
+++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++
Processing sub-CON02 in ses-control:
---------------------------------------------
Brain extracted succesfully
Registered to MNI152 succesfully
Segmented tissues succesfully
Subcortical structures segmented succesfully
+++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++
Processing sub-CON06 in ses-control2:
---------------------------------------------
Brain extracted succesfully
Registered to MNI152 succesfully
Segmented tissues succesfully
Subcortical structures segmented succesfully
+++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++
Processing sub-CON07 in ses-control:
-----------------

In the case of DIPY the addition of subcortical structures to the anatomical mask is not needed. However, if you look carefully, you will see that subcortical regions (e.g., thalamus and/or striatum) are classified as a mixture of white and grey matter. It's ok, they are still good segmentations and "fast" is a good software. Furthermore, lotsof studies do not consider connectivity in the subcortex. Yet, a slightly more accurate pipeline is provided in MRtrix3by the command 5ttgen fsl (https://mrtrix.readthedocs.io/en/latest/reference/commands/5ttgen.html#ttgen-fsl). It implements a quite robust iteration over "fast" and "first" and combines the outputs into a single image. Hence, you need FSL and MRtrix3 to execute it.

## Pre-processing of diffusion weighted images (dwi)

Diffusion MRI is slightly more intricate than anatomical scans. The physical phenomena behind the captured signal and further fiber reconstruction demand careful understanding of what is a true process happening inside the brain and what is induced by the operations during the acquisition. For that, we have no space to go into each detail that goes beyond a simple input and pre-processed output.

In order to avoid you some crazy days in the search of the holy grail, we drop this piece of code to help you with the workflow, specific steps and information required by each one of them. Note how depending on the date that the data was acquired, some of the steps would not be applicable due to lack of information and acquisitions.

A bit of a disclaimer... If some of this was useful for your purposes, consider acknowledging the work behind it:
```
@article{fmridtitumor2022,
     title={Structural Reorganization Following a Brain Tumor: A Machine Learning Study Considering Desynchronized Functional Oscillations},
     author={Falc{\'o}-Roget, Joan and Sambataro, Fabio and Cacciola, Alberto and Crimi, Alessandro},
     journal={bioRxiv},
     pages={2022--11},
     year={2022},
     publisher={Cold Spring Harbor Laboratory}
}
```

### General workflow and steps to be followed

```
import logging
import shutil
import os

from utils.paths import *
from utils.steps import *
from utils.acq_index_files import find_secondary_acquisitions

def dwi_steps(f, output_directory, whole_path, subjectID, session, mri, name, acq, weight, extension, config):
    """
    Performs the preprocessing steps in diffusion weighted images.
    """
    output_directory = output_directory + 'co-registered/'
    logging.info(f" PreProcessing {subjectID} in session {session} with mri {mri}")
    steps = []

    # Relevant paths
    file_dir = output_directory+subjectID+'/'+session+'/'
    mri_path = bids_tree(file_dir, mris=config["subjects"]["mris"])[mri]
    int_path, at_path = mri_path+'intermediate/', file_dir+'atlas/'
    sec_path = mri_path+'intermediate/secondary/'
    check_path(int_path)
    check_path(sec_path)

    ### Topup Susceptiility Artifacts Estimation ###
    sec_files, available_secs = find_secondary_acquisitions(whole_path, subjectID, session, acq, weight)
    if available_secs:
        # Prepare data for topup and eddy
        b0_vols, topup_data = prepare_topup(
            input=f, tmp_dir=int_path, raw_dir=whole_path, sec_files=sec_files, skip=config["data"]["skip"], vols=1
        )
        int_output, eddy_acq, eddy_index, bvecs, bvals = prepare_eddy(
            input=f, tmp_dir=int_path, raw_dir=whole_path, sec_files=sec_files, merge=config["merge_acqs"]
            )

        # TopUp can be performed only if there are at least one secondary acquisitions
        unwarped, unwarped_mask = topup(input=b0_vols, output=sec_path+'topup', datain=topup_data, skip=config["data"]["skip"])
        steps.append("Top Up off-resonance field map")

        logging.info(f" {subjectID} Suceptibility artifacts done")
    else:
        # No susceptibility - then prepare for eddy
        int_output, eddy_acq, eddy_index, bvecs, bvals = prepare_eddy(input=f, tmp_dir=int_path, raw_dir=whole_path)

    # Brain Extraction 
    int_output, mask = brain_extraction(input=int_output, output=int_path+name+'_bet', skip=config["data"]["skip"], method=config["dwi_be"])
    steps.append("Brain Extraction")
    logging.info(f" {subjectID} Brain extraction done")

    # Denoise
    int_output = MP_PCA_denoise(input=int_output, output=int_path+name+'_mppca', skip=config["data"]["skip"], mask=mask)
    steps.append("MP_PCA_denoise")
    logging.info(f" {subjectID} Denoising done")

    # Gibbs artifacts
    int_output = gibbs_unringing(input=int_output, output=int_path+name+'_gibbs', skip=config["data"]["skip"])
    steps.append("Gibbs_unringing")
    logging.info(f" {subjectID} Gibbs artifacts removal done")

    # Eddy + Motion + Bmatrix Correction 
    tpd = sec_path+'topup' if available_secs else None
    ed_mask = unwarped_mask if available_secs else mask
    int_output, rotated_bvecs = eddy(
        input=int_output, output=int_path+name+'_eddy', directory=sec_path, acq=eddy_acq, index=eddy_index,
        bvecs=bvecs, bvals=bvals, mask=ed_mask, skip=config["data"]["skip"], topup_data=tpd, cuda=config["eddy"]["cuda"], version=config["eddy"]["version"]
    )
    steps.append("Eddy-Motion-B correction")
    logging.info(f" {subjectID} Eddy, Motion and Bmatrix correction done")

    # Remove eddy added background 
    int_output, mask = brain_extraction(input=int_output, output=int_path+name+'_eddy_bet', skip=config["data"]["skip"], method=config["dwi_be"])
    steps.append("Brain Extraction")
    logging.info(f" {subjectID} Eddy background removal done")

    # Registration // creation of reference+upsample
    if config["reference"] == 'dwi':
        if config["reference"] != "atlas":
            check_path(at_path)
        
        # Copy bval and rename bvec to main derivatives folder 
        shutil.copyfile(rotated_bvecs, mri_path+name+'.bvec')
        shutil.copyfile(whole_path+name+'.bval',mri_path+name+'.bval')

        # Reference is DWI
        int_output = upsample(input=int_output, output=mri_path+name, skip=config["data"]["skip"], factor=config["vox_size"]["dwi"])
        upsample(input=mask, output=mri_path+name+'_mask', skip=config["data"]["skip"], factor=config["vox_size"]["dwi"])
        steps.append("Upsampled to %s mm" % config["vox_size"]["dwi"])

        reference_img = create_reference(int_output, output=at_path+name+'_ref', skip=config["data"]["skip"])
        steps.append("Created 3D DWI reference image")
    elif config["reference"] == 'anat':
        # Reference is Anat
        reference_img = file_dir+'anat/'+subjectID+'_'+session+'_'+'T1w.nii.gz'
        int_output, tr_matrix = register_to_reference(
            input=int_output, output=int_path+name+'_reg', reference=reference_img, skip=config["data"]["skip"], dim=4
        )
        reg_mask, _ = register_to_reference(
            input=mask, output=int_path+name+'_reg_mask', reference=reference_img, skip=config["data"]["skip"], matrix=tr_matrix
        )
        steps.append("Registered to Anat")

        # B-Matrix rotation
        rotate_Bmatrix(bvecs=rotated_bvecs, bvals=bvals, output=mri_path+name, matrix=tr_matrix)
        steps.append("BMatrix Reorientation")

        # Copy the transformation matrix to the output directory
        shutil.copyfile(tr_matrix, mri_path+name+'_transform.txt') 

        # Upsample to dwi resolution
        int_output = upsample(input=int_output, output=mri_path+name, skip=config["data"]["skip"], factor=config["vox_size"]["dwi"])
        upsample(input=reg_mask, output=mri_path+name+'_mask', skip=config["data"]["skip"], factor=config["vox_size"]["dwi"])
        steps.append("Upsampled to %s mm" % config["vox_size"]["dwi"])
    elif config["reference"] == 'atlas':
        # TODO: Need further testing, for some reason the output is filled with zeros!

        # Reference is Atlas
        reference_img = config["atlas"]["REF"]
        int_output, tr_matrix = register_to_reference(
            input=int_output, output=mri_path+name, reference=reference_img, skip=config["data"]["skip"], dim=4
        )
        reg_mask, _ = register_to_reference(
            input=mask, output=mri_path+name+'_mask', reference=reference_img, skip=config["data"]["skip"], matrix=tr_matrix
        )
        steps.append("Registered to Atlas")

        # B-Matrix rotation
        rotate_Bmatrix(bvecs=rotated_bvecs, bvals=bvals, output=mri_path+name, matrix=tr_matrix)
        steps.append("BMatrix Reorientation")
        
        # Copy the transformation matrix to the output directory
        shutil.copyfile(tr_matrix, mri_path+name+'_transform.txt') 

        # Upsample to dwi resolution
        int_output = upsample(int_output, output=mri_path+name, skip=False, factor=config["vox_size"]["dwi"])
        upsample(input=reg_mask, output=mri_path+name+'_mask', skip=False, factor=config["vox_size"]["dwi"])
        steps.append("Upsampled to %s mm" % config["vox_size"]["dwi"])
    else:
        raise ValueError("Reference image not recognized")
    logging.info(f" {subjectID} Registration steps done")


    # Delete concatenated from original
    os.remove(bvecs)
    os.remove(bvals)

    # Copy json to derivatives
    shutil.copyfile(whole_path+name+'.json',mri_path+name+'.json')

    # Keep intermediate
    if not config["data"]["keep_intermediate"]:
        os.system(f"rm -rf {int_path}")

    ### Output summary ###
    logging.info(f" Subject {subjectID} in session {session} with mri {mri} \n with completed preprocessing steps: {', '.join(steps)}")

if __name__ == "__main__":
    pass
```

### Description and implementation of each step

```
import os
from subprocess import Popen, STDOUT, PIPE
from pathlib import Path
import shutil

from utils.acq_index_files import acq_file, index_file
from utils.paths import check_path

from dipy.segment.mask import median_otsu 
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table, reorient_bvecs
from nibabel import load, Nifti1Image, save
import numpy as np

def brain_extraction(input, output, skip, method=None):
    """
    Skull stripping using FSL's bet2 (for anatomical) and DIPY's median otsu (for diffusion).
    """
    mask = output + '_mask.nii.gz'
    if skip and os.path.exists(mask) and os.path.exists(output+'.nii.gz'):
        return output+'.nii.gz', mask
    else: 
        if 'T1w' in input:
            os.system(
                f"bet2 {input} {output} -f 0.5 -g -.6"
            )
            os.system(
                f"bet2 {output} {output} -f 0.35 -m"
            )
        else:
            if method == "median_otsu":
                img = load(input)
                data = img.get_fdata()
                vol_idx = range(len(data[0,0,0,:])) if len(data.shape)==4 else None
                bet_data, mask_data = median_otsu(data, vol_idx=vol_idx, numpass=5, median_radius=5, dilate=3, autocrop=False) 
                # Cropping the mask introduces dimensionality issues in eddy when using the unwarped masked b0 volume
                save(Nifti1Image(bet_data, img.affine, img.header), output+'.nii.gz')
                save(Nifti1Image(mask_data, img.affine, img.header), mask)
            elif method == "bet2":
                os.system( # Extract the first volume
                    f"fslroi {input} {output}_b0volume 0 1"
                )
                os.system( # Mask the first volume
                    f"bet2 {output}_b0volume {output} -f 0.19 -m"
                )
                os.system( # Apply the mask to all the volumes
                    f"fslmaths {input} -mas {mask} {output}"
                )
            elif method is None:
                print("--- No brain extraction method is provided ---")
            else:
                raise ValueError("Brain Extraction method not implemented")
        return output+'.nii.gz', mask

def MP_PCA_denoise(input, output, mask, skip):
    """
    Marchenko-Pasteur PCA denoising using MRtrix3.
    """
    if skip and os.path.exists(output+'.nii.gz'):
        return output+'.nii.gz' 
    else:
        os.system(
            f"dwidenoise {input} {output}'.nii.gz' -mask {mask} -force -quiet"
        )
        return output+'.nii.gz' 

def gibbs_unringing(input, output, skip):
    """
    Removal of Gibbs ringing artifacts using MRtrix3.
    """
    if skip and os.path.exists(output+'.nii.gz'):
        return output+'.nii.gz' 
    else:
        os.system(
            f"mrdegibbs {input} {output}'.nii.gz' -force -quiet"
        )
        return output+'.nii.gz' 

def register_to_reference(input, output, skip, reference, matrix=None, dim=3):
    """
    #TODO: Add description
    """
    if skip and os.path.exists(output+'.nii.gz'):
        return output+'.nii.gz', output + '_transform.txt'
    else:        
        if dim == 3:
            if matrix is None: # Find the linear transformation
                transform_mat = output + '_transform.txt'
                os.system(
                    f"flirt -in {input} -ref {reference} -out {output} -omat {transform_mat}"
                )
                return output+'.nii.gz', transform_mat
            else: # Apply the transformation
                os.system(
                    f" flirt -in {input} -ref {reference} -out {output} -applyxfm -init {matrix}"
                )
                return output+'.nii.gz', matrix
        elif dim == 4:
            if matrix is None: # Find the linear transformation
                # It is assumed that the 4D image has been corrected for motion artifacts so all slices are coregistered!
                tempdir = 'tmp-reg_' + output.split('/')[-1] + '/' #+output.split('/')[-4]+'/'
                check_path(tempdir)
                # Split allthe volumes into separate 3D files
                os.system(
                    f"fslsplit {input} {tempdir} -t"
                )
                transform_mat = output + '_transform.txt'
                # Find the transformation with the first volume
                os.system(
                        f"flirt -in {tempdir}'0000.nii.gz' -ref {reference} -out {tempdir}'0000.nii.gz' -omat {transform_mat}"
                    )
                # Apply the transformation to the rest of the volumes
                for vol in os.listdir(tempdir):
                    if vol != '0000.nii.gz':
                        os.system(
                            f" flirt -in {tempdir+vol} -ref {reference} -out {tempdir+vol} -applyxfm -init {transform_mat}"
                        )
                # Merge all the volumes
                os.system(
                    f"fslmerge -t {output} {tempdir}*.nii.gz"
                )
                # Remove the temporary directory
                os.system(
                    f"rm -rf {tempdir}"
                )
                return output+'.nii.gz', transform_mat
            else: #Apply the transformation
                os.system(
                    f" flirt -in {input} -ref {reference} -out {output} -applyxfm -init {matrix}"
                )
                return output+'.nii.gz', matrix
        else:
            raise ValueError('Dimension must be either 3 or 4')

def bias_correction(input, output, skip, mask):
    """
    Bias correction using N4 ants Algorithm.
    """
    if skip and os.path.exists(output+'.nii.gz'):
        return output+'.nii.gz'
    else:
        os.system( 
            f"N4BiasFieldCorrection -i {input} -o {output}'.nii.gz' -d 3 -x {mask} -r"
        )
        return output+'.nii.gz'

def upsample(input, output, skip, factor=1.5):
    """
    MRtrix3 upsampmling image to increase/decrease resolution.
    """
    if skip and os.path.exists(output+'.nii.gz'):
        return output+'.nii.gz'
    else:
        os.system( 
            f"mrgrid {input} regrid {output}'.nii.gz' -force -quiet -voxel {factor}"
        )
        return output+'.nii.gz'

def create_reference(input, output, skip):
    """
    Create a template image for registration. Useful for registering to dwis, since anatomical
        and atlases are already 3D images.
    """
    if skip and os.path.exists(output+'.nii.gz'):
        return output+'.nii.gz'
    else:
        os.system(
             f"fslroi {input} {output} 0 1"
            )
        return output+'.nii.gz'
def extract_b0s(inputs, output, vols=None):
    """
    Extract the b0 volumes from a DWI image.
    Returns the number of b0 volumes extracted.
    Saves the b0 volumes in the directory specified.
    Options: Extract the specified number of b0 values if vols is not None
    """
    # Extract all b0 volumes
    os.system(
        f"dwiextract {inputs[0]} {output} -bzero -fslgrad {inputs[2]} {inputs[1]} -force -quiet"
    )
    # Keep the specified number of b0 volumes
    if vols is not None:
        os.system(
            f"mrconvert {output} {output} -coord 3 0:{vols-1} -force -quiet"
        )
    # Report the final number of volumes extracted
    message = Popen(f"fslnvols {output}", shell=True, stdout=PIPE).stdout.read()
    Nvolumes = int(str(message).removeprefix('b\'').removesuffix('\'').removesuffix('\\n').split('\\n')[0])
    return output, Nvolumes

def prepare_topup(input, tmp_dir, sec_files, raw_dir, skip, vols=None):
    """
    Prepare the images and files to be used in the topup step.
    ###
    # TODO: Improve what is said below
    Here it is assumed that all the acquisitions are co-registered.
        Might not be exactly true, but for these datasets it approximately holds.
        The fact that this is not exactly true will cause a WARNING whehn merging the volumes
        from different acquisitions.
    ####    
    """
    output = tmp_dir+input.split('.')[0].split('/')[-1]+'_b0vols'
    if skip and os.path.exists(output+'.nii.gz') and os.path.exists(tmp_dir+'topup_data.txt'):
        return output+'.nii.gz', tmp_dir+'topup_data.txt'
    else:
        # List of main files in the scheme
        main_file = input.split('.')[0]  
        main_files = [main_file+'.nii.gz', main_file+'.bval', main_file+'.bvec', main_file+'.json']
        ### These two lines are useful if and only if a bet has been run at the very begining of the pipeline
        #origin_name = raw_dir+'_'.join(input.split('.')[0].split('/')[-1].split('_')[:-1])
        #main_files = [main_file+'.nii.gz', origin_name+'.bval', origin_name+'.bvec', origin_name+'.json']

        # Extract b0 volumes from main file
        jsons, suffix = [], '_b0s'
        _, Nvs = extract_b0s(main_files, tmp_dir+'main'+suffix+'.nii.gz', vols=vols)
        for i in range(Nvs):
            jsons.append(main_files[3]) # append the jsons as many times a b0s

        # Extract b0 volumes from secondary files
        for i in range(len(sec_files)):
            _, Nvs = extract_b0s(sec_files[i], tmp_dir+'sec'+str(i)+suffix+'.nii.gz', vols=vols)
            for j in range(Nvs):
                jsons.append(sec_files[i][3]) # append the jsons as many times a b0s

        # Merge the b0 files
        os.system(
            f"fslmerge -t {output} {tmp_dir}*{suffix}.nii.gz"
        )
        os.system(
            f"rm {tmp_dir}*{suffix}.nii.gz"
        )
        # Create the acqparams file for the --datain option in topu up
        acq_file(jsons, tmp_dir+'topup_data.txt')
        return output+'.nii.gz', tmp_dir+'topup_data.txt'

        
def topup(input, output, datain, skip):
    """
    https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/topup/TopupUsersGuide
    Runs topup fsl command to estimate the off-ressonance field.
    Additionally it performs a brain extraction on the unwarped b0 volumes to output a mask for eddy.
    This function needs prepare_topup to be run first to run topup on the bo volumes from all the acquisitions.
    Returns the unwarped b0 volumes and the mask estimated from them to pass as input to eddy (suggested in FSL wiki).
    """
    if skip and os.path.exists(output+'_movpar.txt') and os.path.exists(output+'_fieldcoef.nii.gz') and os.path.exists(output+'_unwarped.nii.gz'):
        return output+'_unwarped.nii.gz', output+'_unwarped_mask.nii.gz'
    else:
        os.system(
            f"topup --imain={input} --datain={datain} --out={output} --iout={output}_unwarped"
        )
        os.system(
            f"fslmaths {output}_unwarped -Tmean {output}_unwarped"
        )
        b0s_unwarped, b0s_unwarped_mask = brain_extraction(input=output+'_unwarped.nii.gz', output=output+'_unwarped', skip=False, method="bet2")
        os.system(
            f"rm {'/'.join(datain.split('/')[:-1])}/*.topup_log"
        )
        return b0s_unwarped, b0s_unwarped_mask

def concatenate_gradients(bvals, bvecs, output):
    """
    Concatenate bvals and bvecs from different acquisitions into a single file.
    """
    cat_bvals = '_'.join(output.split('_')[:-1])+'_concat.bval'
    cat_bvecs = '_'.join(output.split('_')[:-1])+'_concat.bvec'

    # Read and concatenate bvals and bvecs
    bval_lines, bvec_lines = '', ['', '', '']
    for i, j in zip(bvals, bvecs):
        with open(i, 'r') as bval_table:
            current = bval_table.readlines()[0].strip()
            bval_lines = bval_lines + current + ' ' 
        with open(j, 'r') as bvec_table:
            current = bvec_table.readlines()
            bvec_lines[0] = bvec_lines[0] + current[0].strip() + ' '
            bvec_lines[1] = bvec_lines[1] + current[1].strip() + ' ' 
            bvec_lines[2] = bvec_lines[2] + current[2].strip() + ' ' 
    
    # Write the concatenated bvals and bvecs
    with open(cat_bvals, 'w') as bval_table:
        bval_table.write(bval_lines)
    with open(cat_bvecs, 'w') as bvec_table:
        bvec_table.write('\n'.join(bvec_lines))
    return cat_bvecs, cat_bvals

def prepare_eddy(input, tmp_dir, raw_dir, sec_files=None, merge=None):
    """
    Prepares the images and files to be used in the eddy step.
    Based on the configuration and the secondary acquisitions,
        it will merge everythin sequentially and the index file
        will be created accordingly. Feel free to merge with other schemes.

    ###
    # TODO: Improve what is said below
    Here it is assumed that all the acquisitions are co-registered.
        Might not be exactly true, but for these datasets it approximately holds.
        The fact that this is not exactly true will cause a WARNING whehn merging the volumes
        from different acquisitions.
    ####
    """
    eddy_acq = tmp_dir+'eddy_acq.txt'
    eddy_index = tmp_dir+'eddy_ind.txt'

    # List of main files in the scheme
    main_file = input.split('.')[0]    
    files = [[main_file+'.nii.gz', main_file+'.bval', main_file+'.bvec', main_file+'.json']]
    ### These two lines are useful if and only if a bet has been run at the very begining of the pipeline
    #origin_name = raw_dir+'_'.join(input.split('.')[0].split('/')[-1].split('_')[:-1])
    #files = [[main_file+'.nii.gz', origin_name+'.bval', origin_name+'.bvec', origin_name+'.json']]

    # Merge files if specified
    if merge and sec_files is not None:
        merged_file = tmp_dir+'merged_acquisitions.nii.gz'
        os.system(f"cp {input} {merged_file}")
        for i in range(len(sec_files)):
            files.append(sec_files[i])
            os.system(
                f"fslmerge -t {merged_file} {merged_file} {sec_files[i][0]}"
            )
    else:
        merged_file = input

    # Create the acquisition and index parameters file
    jsons, niis = [files[i][-1] for i in range(len(files))], [files[i][0] for i in range(len(files))]
    bvals, bvecs = [files[i][1] for i in range(len(files))], [files[i][2] for i in range(len(files))]
    acq_file(jsons, eddy_acq)
    index_file(niis, eddy_index)
    cat_bvecs, cat_bvals = concatenate_gradients(bvals, bvecs, main_file)
    return merged_file, eddy_acq, eddy_index, cat_bvecs, cat_bvals


def eddy(input, output, directory, acq, index, bvecs, bvals, mask, skip, topup_data=None, cuda=False, version=None):
    """
    https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/eddy/UsersGuide
    Runs eddy, motion and b-matrix correction according to previous steps (prepare_topup, topup, prepare_eddy).
    cuda: controls which version of eddy is used
    """
    basename = directory + 'eddy_output'
    if skip and os.path.exists(output+'.nii.gz') and os.path.exists(output+'.bvec'):
        return output+'.nii.gz', output+'.bvec'
    else:
        ed_type = 'cuda'+str(version) if cuda else 'openmp'
        if topup_data is None:
            os.system(
                f"eddy_{ed_type} --imain={input} --mask={mask} --acqp={acq} --index={index} --bvecs={bvecs} --bvals={bvals} --out={basename}"
            )
        else:
            os.system(
                f"eddy_{ed_type} --imain={input} --mask={mask} --acqp={acq} --index={index} --bvecs={bvecs} --bvals={bvals} \
                --topup={topup_data} --out={basename}" 
            )

        # Filtering and renaming the important output files
        Path(basename+'.nii.gz').rename(output+'.nii.gz')
        Path(basename+'.eddy_rotated_bvecs').rename(output+'.bvec')
        return output+'.nii.gz', output+'.bvec'

def rotate_Bmatrix(bvecs, bvals, output, matrix):
    """
    Rotates the b-matrix after the diffusion data has been 'moved' to the reference image.
    It assumes that all volumes are corrected for motion artifacts and are co-registered;
        therefore, the same transformation (rotation) matrix is applied to all b-vectors.
    """
    # Bvals are not rotated
    shutil.copyfile(bvals, output+'.bval')

    # Read and rotate non-zeros bvecs
    b_vals, b_vecs = read_bvals_bvecs(bvals, bvecs)
    gtab = gradient_table(b_vals, b_vecs)
    vols = gtab.bvals.shape[0]
    v_trans, trans = np.loadtxt(matrix), list()
    for v in range(vols):
        if not gtab.b0s_mask[v]:
            trans.append(v_trans)
    gtab_corr = reorient_bvecs(gtab, trans)
    np.savetxt(output+'.bvec', gtab_corr.bvecs.T)
    return output+'.bvec', output+'.bval'

```